In [1]:
# Autoreload possibly interferes with IntelliJ debugging
%reload_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
log = lambda msg: logging.info(msg)


# Post-human pipeline (single file)

In [2]:
import pandas as pd
import os

input_dir = "C:\\dev\\play\\brainwave-data"
stats_df = pd.read_csv(input_dir + os.path.sep + "stats.csv")

# Post-human processing
This requires user interaction first to provide sleep times etc.

In [7]:
from sleep_events import load_days_data
from sleep_events import pimp_my_days_data

days_data = load_days_data()
days_data = pimp_my_days_data(days_data)

In [6]:
import run_feature_pipeline
from run_post_human_pipeline import cached_post_human_pipeline

#dir_name = "2024-11-01-21-32-22"
dir_name = "2024-11-08-21-16-53"
input_file = f"C:\\dev\\play\\brainwave-data\\{dir_name}\\raw.fif"
yasa_df = run_feature_pipeline.cached_pipeline(log, input_file, stats_df)
#post_human_df = cached_post_human_pipeline(log, dir_name, input_file, stats_df, days_data, yasa_df)
#post_human_df

2024-11-21 17:13:54,880 - INFO - Loading cached file C:\dev\play\brainwave-data\2024-11-08-21-16-53\raw.with_features.csv


# Replicate the pipeline

In [3]:
from models.eeg_states.eeg_states import load_and_prepare_tired_wired_eeg_state_events

eeg_state_events = load_and_prepare_tired_wired_eeg_state_events()


C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:115: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-10-09 20:17:27.992000+00:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  events.at[i, 'since'] = since_time
C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:117: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  events['duration'] = events['timestamp'] - events['since']


In [4]:
eeg_state_events

,event,version,phase,note,duration,TimestampUK,SinceUK,dayAndNightOf
0,tired_long,NaN,NaN,NaN,0 days 00:05:00,2024-10-09 21:22:27.992000+01:00,2024-10-09 21:17:27.992000+01:00,2024-10-09
1,tired,NaN,NaN,NaN,NaT,2024-10-09 21:28:57.046000+01:00,NaT,2024-10-09
2,tired,NaN,NaN,NaN,NaT,2024-10-09 21:32:36.824000+01:00,NaT,2024-10-09
3,tired_long,NaN,NaN,NaN,0 days 00:00:15.951000,2024-10-09 21:32:52.775000+01:00,2024-10-09 21:32:36.824000+01:00,2024-10-09
4,tired,NaN,NaN,NaN,NaT,2024-10-09 21:33:13.921000+01:00,NaT,2024-10-09
...,...,...,...,...,...,...,...,...
437,sleepy_long,v1,settling,NaN,0 days 00:00:57.114000,2024-11-20 21:33:35.168000+00:00,2024-11-20 21:32:38.054000+00:00,2024-11-20
438,tired,NaN,NaN,NaN,NaT,2024-11-20 21:39:45.419000+00:00,NaT,2024-11-20
439,sleepy,NaN,NaN,NaN,NaT,2024-11-20 21:41:05.764000+00:00,NaT,2024-11-20
440,sleepy,NaN,NaN,NaN,NaT,2024-11-20 21:43:13.344000+00:00,NaT,2024-11-20


In [8]:
from catboost import CatBoostClassifier
from models.eeg_states.eeg_states import add_event_type
from run_post_human_pipeline import add_periods
from sleep_events import convert_timestamps_to_uk

convert_timestamps_to_uk(yasa_df, 'Timestamp', 'TimestampUK')
yasa_df = add_periods(log, dir_name, input_file, stats_df, days_data, yasa_df)
add_event_type(yasa_df, eeg_state_events)

100%|██████████| 1138/1138 [00:00<00:00, 2527.93it/s]


In [9]:
yasa_df['dayAndNightOf'].info()

<class 'pandas.core.series.Series'>
Index: 1138 entries, 0 to 1137
Series name: dayAndNightOf
Non-Null Count  Dtype 
--------------  ----- 
1138 non-null   object
dtypes: object(1)
memory usage: 50.1+ KB


In [10]:
before_ready_to_sleep = yasa_df[yasa_df['60MinsBeforeReadyToSleep'] == 1]
before_ready_to_sleep.shape

(120, 463)

In [11]:
before_ready_to_sleep

,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,...,minsSinceAsleep,15MinsBeforeReadyToSleep,30MinsBeforeReadyToSleep,60MinsBeforeReadyToSleep,DuringReadyToSleep,minsSinceReadyToSleep,minsSinceGotIntoBed,minsUntilWake,epoch_type,matched_event
epoch,,,,,,,,,,,,,,,,,,,,,
231,W,0.969882,231,2024-11-08 23:12:23.214781046+00:00,Fpz-M1,W,0.969882,0,3.240352e-10,-0.003014,...,-63.113087,0,0,1,0,-59.913087,112.570246,451.113087,None,None
232,W,0.977515,232,2024-11-08 23:12:53.214781046+00:00,Fpz-M1,W,0.977515,0,2.386019e-10,-0.007841,...,-62.613087,0,0,1,0,-59.413087,113.070246,450.613087,None,None
233,W,0.982693,233,2024-11-08 23:13:23.214781046+00:00,Fpz-M1,W,0.982693,0,4.606290e-10,-0.009934,...,-62.113087,0,0,1,0,-58.913087,113.570246,450.113087,None,None
234,W,0.955983,234,2024-11-08 23:13:53.214781046+00:00,Fpz-M1,W,0.955983,0,2.718269e-10,-0.011219,...,-61.613087,0,0,1,0,-58.413087,114.070246,449.613087,None,None
235,W,0.964417,235,2024-11-08 23:14:23.214781046+00:00,Fpz-M1,W,0.964417,0,2.762543e-10,-0.012228,...,-61.113087,0,0,1,0,-57.913087,114.570246,449.113087,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,W,0.478501,346,2024-11-09 00:09:53.214781046+00:00,Fpz-M1,W,0.478501,0,2.124091e-10,-0.051155,...,-5.613087,1,1,1,0,-2.413087,170.070246,393.613087,None,None
347,N2,0.762124,347,2024-11-09 00:10:23.214781046+00:00,Fpz-M1,N2,0.762124,2,8.303643e-11,-0.046269,...,-5.113087,1,1,1,0,-1.913087,170.570246,393.113087,None,None
348,W,0.918860,348,2024-11-09 00:10:53.214781046+00:00,Fpz-M1,W,0.918860,0,9.917654e-10,-0.041469,...,-4.613087,1,1,1,0,-1.413087,171.070246,392.613087,None,None


In [15]:
from models.eeg_states.eeg_states_model import predict_only_tired_vs_wired_model_pipeline

models_and_data = [predict_only_tired_vs_wired_model_pipeline('main', before_ready_to_sleep, False)]
model = CatBoostClassifier()
model.load_model("models/eeg_states/main_catboost_model.cbm")
predictions = model.predict_proba(models_and_data[0].X)
predictions_df = pd.DataFrame(predictions, index=models_and_data[0].X.index)
predictions_df['TiredVsWired'] = predictions_df[1]
predictions_df.drop([0, 1], axis=1, inplace=True)

['Main_eeg_abspow_s', 'Main_eeg_abspow_c7min_norm_s', 'Main_eeg_abspow_p2min_norm_s', 'Main_eeg_alpha_s', 'Main_eeg_alpha_c7min_norm_s', 'Main_eeg_alpha_p2min_norm_s', 'Main_eeg_alphaaa_s', 'Main_eeg_alphaaa_c7min_norm_s', 'Main_eeg_alphaaa_p2min_norm_s', 'Main_eeg_alphaab_s', 'Main_eeg_alphaab_c7min_norm_s', 'Main_eeg_alphaab_p2min_norm_s', 'Main_eeg_alphaabs_s', 'Main_eeg_alphaabs_c7min_norm_s', 'Main_eeg_alphaabs_p2min_norm_s', 'Main_eeg_alphaabsaa_s', 'Main_eeg_alphaabsaa_c7min_norm_s', 'Main_eeg_alphaabsaa_p2min_norm_s', 'Main_eeg_alphaabsab_s', 'Main_eeg_alphaabsab_c7min_norm_s', 'Main_eeg_alphaabsab_p2min_norm_s', 'Main_eeg_at_s', 'Main_eeg_at_c7min_norm_s', 'Main_eeg_at_p2min_norm_s', 'Main_eeg_auc_s', 'Main_eeg_auc_c7min_norm_s', 'Main_eeg_auc_p2min_norm_s', 'Main_eeg_beta_s', 'Main_eeg_beta_c7min_norm_s', 'Main_eeg_beta_p2min_norm_s', 'Main_eeg_betaabs_s', 'Main_eeg_betaabs_c7min_norm_s', 'Main_eeg_betaabs_p2min_norm_s', 'Main_eeg_db_s', 'Main_eeg_db_c7min_norm_s', 'Main_eeg_

In [41]:
predictions_df

,TiredVsWired
epoch,
28,0.315348
29,0.432130
30,0.751347
31,0.414552
32,0.311561
...,...
143,0.030932
144,0.024494
145,0.140188


In [39]:
predictions_df

,0,1,TiredVsWired
epoch,,,
28,0.684652,0.315348,0.315348
29,0.567870,0.432130,0.432130
30,0.248653,0.751347,0.751347
31,0.585448,0.414552,0.414552
32,0.688439,0.311561,0.311561
...,...,...,...
143,0.969068,0.030932,0.030932
144,0.975506,0.024494,0.024494
145,0.859812,0.140188,0.140188


In [42]:
yasa_df_with_predictions = pd.concat([yasa_df, predictions_df], axis=1)

In [43]:
yasa_df_with_predictions

,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,...,15MinsBeforeReadyToSleep,30MinsBeforeReadyToSleep,60MinsBeforeReadyToSleep,DuringReadyToSleep,minsSinceReadyToSleep,minsSinceGotIntoBed,minsUntilWake,epoch_type,matched_event,TiredVsWired
epoch,,,,,,,,,,,,,,,,,,,,,
0,W,0.999395,0,2024-11-07 21:26:18.485414982+00:00,Fpz-M1,W,0.999395,0,4.993414e-09,3.034542,...,0,0,0,0,-73.69191,-1.69191,597.49191,wired,210,NaN
1,W,0.999047,1,2024-11-07 21:26:48.485414982+00:00,Fpz-M1,W,0.999047,0,4.687774e-08,2.991824,...,0,0,0,0,-73.19191,-1.19191,596.99191,None,None,NaN
2,W,0.999430,2,2024-11-07 21:27:18.485414982+00:00,Fpz-M1,W,0.999430,0,1.161335e-08,2.729585,...,0,0,0,0,-72.69191,-0.69191,596.49191,None,None,NaN
3,W,0.999343,3,2024-11-07 21:27:48.485414982+00:00,Fpz-M1,W,0.999343,0,2.888595e-08,2.509642,...,0,0,0,0,-72.19191,-0.19191,595.99191,None,None,NaN
4,W,0.998353,4,2024-11-07 21:28:18.485414982+00:00,Fpz-M1,W,0.998353,0,2.170471e-08,2.220646,...,0,0,0,0,-71.69191,0.30809,595.49191,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,W,0.995203,1240,2024-11-08 07:46:18.485414982+00:00,Fpz-M1,W,0.995203,0,1.276919e-09,0.168469,...,0,0,0,0,546.30809,618.30809,-22.50809,None,None,NaN
1241,W,0.997428,1241,2024-11-08 07:46:48.485414982+00:00,Fpz-M1,W,0.997428,0,7.675433e-10,0.161403,...,0,0,0,0,546.80809,618.80809,-23.00809,None,None,NaN
1242,W,0.996014,1242,2024-11-08 07:47:18.485414982+00:00,Fpz-M1,W,0.996014,0,9.583334e-10,0.156325,...,0,0,0,0,547.30809,619.30809,-23.50809,None,None,NaN


In [45]:
yasa_df_with_predictions["TiredVsWired"].mean()

0.5650632491976203